In [ ]:
!git clone --branch onnx https://github.com/AXKuhta/RWKV-LM
%cd /content/RWKV-LM
!git checkout 648bf3c81af1355343b9f36ee7dba57775c81ead # <-- to ensure this notebook works even if repo is updated

In [ ]:
%cd /content/RWKV-LM/RWKV-v4

In [ ]:
!pip install transformers

In [ ]:
!wget https://huggingface.co/BlinkDL/rwkv-4-pile-169m/resolve/main/RWKV-4-Pile-169M-20220807-8023.pth
MODEL_NAME = 'RWKV-4-Pile-169M-20220807-8023'
n_layer = 12
n_embd = 768
ctx_len = 1024

# !wget https://huggingface.co/BlinkDL/rwkv-4-pile-430m/resolve/main/RWKV-4-Pile-430M-20220808-8066.pth
# MODEL_NAME = 'RWKV-4-Pile-430M-20220808-8066'
# n_layer = 24
# n_embd = 1024
# ctx_len = 1024

# !wget https://huggingface.co/BlinkDL/rwkv-4-pile-1b5/resolve/main/RWKV-4-Pile-1B5-20220814-4526.pth
# MODEL_NAME = 'RWKV-4-Pile-1B5-20220814-4526'
# n_layer = 24
# n_embd = 2048
# ctx_len = 1024

In [9]:
import numpy as np
import torch
import os
from torch.nn import functional as F

from transformers import PreTrainedTokenizerFast

os.environ['RWKV_FLOAT_MODE'] = 'fp32'
os.environ['RWKV_RUN_DEVICE'] = 'cpu'
model_type = 'RWKV'

from src.model_run import RWKV_RNN

np.set_printoptions(precision=4, suppress=True, linewidth=200)

tokenizer = PreTrainedTokenizerFast(tokenizer_file='20B_tokenizer.json')
context = '\nIn a shocking finding,'

model = RWKV_RNN(MODEL_NAME, os.environ['RWKV_RUN_DEVICE'], model_type, n_layer, n_embd, ctx_len)

ctx = torch.randint(5000, (1024,), dtype=torch.int32 ) + 100
xx_att = torch.zeros(n_layer, n_embd)
aa_att = torch.zeros(n_layer, n_embd)
bb_att = torch.zeros(n_layer, n_embd)
pp_att = torch.zeros(n_layer, n_embd) - 1e30
xx_ffn = torch.zeros(n_layer, n_embd)

torch.onnx.export(model, args=(ctx, xx_att, aa_att, bb_att, pp_att, xx_ffn), f="rwkv.onnx", input_names = ["idx", "xx_att", "aa_att", "bb_att", "pp_att", "xx_ffn"], output_names = ["x", "xx_att_r", "aa_att_r", "bb_att_r", "pp_att_r", "xx_ffn_r"], verbose=True)

In [ ]:
# Now we quantize the model:
!pip install onnxruntime
!pip install onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
quantize_dynamic("/content/RWKV-LM/RWKV-v4/rwkv.onnx", "/content/RWKV-LM/RWKV-v4/rwkv-uint8.onnx", weight_type=QuantType.QUInt8, extra_options={"MatMulConstBOnly":False})